In [1]:
import math
import numpy as np
import pandas as pd
from math import sin, cos, atan2, sqrt
from pyproj import Proj, transform

from cv2 import Rodrigues
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import accuracy_score
import lightgbm as lgb

from tqdm import tqdm
import pyproj

from lib.noglobal import noglobal




from external_lib.Predict_with_IMU import execute
from external_lib.Predict_with_IMU  import generate_train_data,generate_test_data,training,ECEF_to_WGS84,WGS84_to_ECEF,prepare_imu_data

from lib.io import load_pickle_data
from external_lib.evaluation_function import calc_haversine

## データ取得

In [2]:
dir_ = "/work/data/input/google-smartphone-decimeter-challenge"
bl_trn_fname = "/work/data/input/google-smartphone-decimeter-challenge/baseline_locations_train.csv"
bl_tst_fname = "/work/data/input/google-smartphone-decimeter-challenge/baseline_locations_test.csv"

sample_fname  = f"{dir_}/sample_submission.csv"

In [3]:
#train_df = pd.read_csv(bl_trn_fname)
#test_df = pd.read_csv(bl_tst_fname)
bl_trn_df = pd.read_csv(bl_trn_fname)
bl_tst_df = pd.read_csv(bl_tst_fname)


## Modeling

### LightGBM

In [10]:
from external_lib.Predict_with_IMU import execute
#bl_trn_df = pd.read_csv(bl_trn_fname)
#bl_tst_df = pd.read_csv(bl_tst_fname)

bl_trn_fname = "/work/data/input/selfmade_dataset/baseline_with_derived_data_v5/train.pkl"
bl_tst_fname = "/work/data/input/selfmade_dataset/baseline_with_derived_data_v5/test.pkl"
bl_trn_df = load_pickle_data(bl_trn_fname)
bl_tst_df = load_pickle_data(bl_tst_fname)

output_bl_trn_fname = "/work/data/submission_list/baseline_/baseline_with_1/train_result.csv"
output_bl_tst_fname = "/work/data/submission_list/baseline_/baseline_with_1/result.csv"
output_bl_trn_df = pd.read_csv(output_bl_trn_fname)
output_bl_tst_df = pd.read_csv(output_bl_tst_fname)

bl_trn_df[["latDeg","lngDeg"]] = output_bl_trn_df[["latDeg","lngDeg"]]
bl_tst_df[["latDeg","lngDeg"]] = output_bl_tst_df[["latDeg","lngDeg"]]




In [48]:
baseline_path = "/work/data/input/google-smartphone-decimeter-challenge/baseline_locations_train.csv"
original_baseline_df = pd.read_csv(baseline_path);

tgt_cns = original_baseline_df["phone"].unique().tolist()[0:21]
tgt_cns = bl_trn_df[bl_trn_df['collectionName'].str.contains("|".join(tgt_cns))]["phone"].unique()


test_target_list = [i for i in bl_tst_df["phone"].unique() if (("SJC-2" in i ) or ("SJC-3" in i )) ]



print(len(train_cols))
#1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21

21


In [11]:

ret_df = execute(bl_trn_df,bl_tst_df,tgt_cns,test_target_list)

100%|██████████| 6/6 [13:34<00:00, 135.75s/it]


Final Dataset shape： (14336, 846)
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Curr

In [39]:
from external_lib.evaluation_function import evaluate_function,calc_haversine

train_df = ret_df[0]
test_df = ret_df[1]

pred_dt = [];
baseline = [];

for key,each_df in train_df[train_df["phone"].str.contains("|".join(tgt_cns))].groupby("phone"):
    
    baseline_tmp_df = bl_trn_df[bl_trn_df["phone"]==key]
    a = calc_haversine(baseline_tmp_df["lngDeg_gt"],baseline_tmp_df["latDeg_gt"],baseline_tmp_df["lngDeg"],baseline_tmp_df["latDeg"])
    #print(key,a.iloc[30:].mean())    
    b = calc_haversine(baseline_tmp_df["lngDeg_gt"],baseline_tmp_df["latDeg_gt"],each_df["lngDeg"],each_df["latDeg"])
    
    print(f"-------------{key}------------------")
    pred_dt = pred_dt + a.iloc[30:].tolist()
    baseline = baseline + b.iloc[30:].tolist()
    
    #print("before",a.iloc[30:].mean())    
    #print("after",b.iloc[30:].mean())

print("before",np.average(pred_dt))
print("after",np.average(baseline))

-------------2021-04-22-US-SJC-1_Pixel4------------------
-------------2021-04-22-US-SJC-1_SamsungS20Ultra------------------
-------------2021-04-28-US-SJC-1_Pixel4------------------
-------------2021-04-28-US-SJC-1_SamsungS20Ultra------------------
-------------2021-04-29-US-SJC-2_Pixel4------------------
-------------2021-04-29-US-SJC-2_SamsungS20Ultra------------------
before 7.316029950727682
after 6.70188907635464


In [55]:
pred = test_df[test_df["phone"]==test_target_list[2]]
pred["label"] = "pred"
val =bl_tst_df[bl_tst_df["phone"] == test_target_list[2]]
val["label"] = "val"
visualize_trafic(pd.concat([pred,val]),"label")

In [31]:

col = "2021-04-22-US-SJC-1_SamsungS20Ultra"

baseline_tmp_df = bl_trn_df[bl_trn_df["phone"]==col]
baseline_tmp_df["label"] = "pred"
pred_tmp_df = train_df[train_df["phone"] == col]
pred_tmp_df["label"] = "baseline"

gt_tmp_df = pred_tmp_df.copy()
gt_tmp_df = gt_tmp_df[gt_tmp_df["phone"] == col]
gt_tmp_df[["latDeg","lngDeg"]] = gt_tmp_df[["latDeg_gt","lngDeg_gt"]]
gt_tmp_df["label"] = "gt"

In [35]:
from external_lib.visualize import visualize_trafic


visualize_trafic(pd.concat([baseline_tmp_df.iloc[30:],pred_tmp_df.iloc[30:],gt_tmp_df.iloc[30:]]),"label")

In [72]:
from external_lib.merge_start_end_points import merge_start_end_points
from external_lib.merge_half_way import merge_half_way
from lib.kalman_filter import generate_kalmanfilter,apply_kalmanfilter

output = bl_tst_df.copy();
tmp_test_df = test_df.copy()
pd_list = [];
for key,each_df in tmp_test_df.groupby("phone"):
    if (key in test_target_list):        
        kf = generate_kalmanfilter();
        
        result = apply_kalmanfilter(each_df[["latDeg","lngDeg"]].to_numpy(),kf);
        each_df[["latDeg","lngDeg"]] = result
        each_df = merge_start_end_points(each_df,merge_flag = True,kalman_flag=False)    
        each_df = merge_half_way(each_df)        
        output.loc[output["phone"] == key,["latDeg","lngDeg"]] = each_df[["latDeg","lngDeg"]]
        pd_list.append(each_df);
    else:
        pass;
        
    
postprosesed_test_df = pd.concat(pd_list).sort_index()

#test_df[test_df["phone"]==test_target_list[0]]

100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


In [67]:
pred = test_df[test_df["phone"]==test_target_list[2]]
pred["label"] = "pred"
val =bl_tst_df[bl_tst_df["phone"] == test_target_list[2]]
val["label"] = "val"
pos = postprosesed_test_df[postprosesed_test_df["phone"] == test_target_list[2]]
pos["label"] = "posted"
visualize_trafic(pd.concat([pred,val,pos]),"label")

In [73]:
test_df = ret_df[1]
sub = pd.read_csv("/work/data/input/google-smartphone-decimeter-challenge/sample_submission.csv");


sub[["latDeg","lngDeg"]] = output[["latDeg","lngDeg"]]

sub.to_csv("./test_saple.csv",index=False)
#test_df